In [2]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
from query_articles import query_articles

In [5]:
API_KEY = os.getenv("NEWS_API_KEY")
query = "Elon Musk AND (trump OR harris OR politics OR political OR election OR PAC OR endorse OR endorsement OR free speech OR censorship OR vote)"
articles = query_articles(query, API_KEY, sort_by="popularity", nb_pages=5)

print(f"Query returned {articles.shape[0]} articles")
print(f"Articles from the following sources were found: {articles['source'].unique()}")
print(f"Here are the first 5 articles:")
print(articles["title"].head())

HTTPError: 429 Client Error:  for url: https://newsapi.org/v2/everything?q=Elon+Musk+AND+%28trump+OR+harris+OR+politics+OR+political+OR+election+OR+PAC+OR+endorse+OR+endorsement+OR+free+speech+OR+censorship+OR+vote%29&apiKey=***REMOVED***&language=en&sortBy=popularity&searchIn=title&page=1